In [26]:
#Use Kafka source for streaming queries 


#!hdfs dfs -mkdir /tmp/iverson/checkpointLocation
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.0,org.apache.spark:spark-streaming-kafka-0-10_2.11:2.4.0'

#!echo 'sc.getConf.get("spark.home")' | spark-shell
#!cd $SPARK_HOME

import findspark
findspark.init(spark_home="/opt/cloudera/parcels/CDH-6.2.1-1.cdh6.2.1.p0.4951328/lib/spark")

KAFKA_TOPIC  = "IZkafkatopic.public.testiz"
KAFKA_BOOTSTRAP_SERVERS_CONS = hname2



customparksession.sparkContext.setLogLevel("WARN")
#customparksession.sparkContext.setLogLevel("ERROR")
#print(" kafka Started ...")
# Construct a streaming DataFrame that reads from iztopics
iz_df = customparksession \
.readStream \
.format("kafka") \
.option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS_CONS) \
.option("subscribe", KAFKA_TOPIC) \
.option("startingOffsets", "earliest") \
.option("max.poll.records", 100)\
.load()

iz_df.printSchema()


#df=iz_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "CAST(topic AS STRING)", "CAST(partition AS STRING)", "CAST(offset AS STRING)", "CAST(timestamp AS STRING)", "CAST(timestampType AS STRING)")
df=iz_df.selectExpr("CAST(key AS STRING) AS key", "to_json(struct(*)) AS value")




root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [ ]:
#Write using Kafka source streaming query output

#readStream
#writeStream
#.format("console")
#.option("startingOffsetsByTimestamp","""{"IZkafkatopic.public.testiz":{"0": 1595882598000, "1": 1595882598000, "2": 1595882598000, "3": 1595882598000, "4": 1595882598000}}""")\
#.option("max.poll.records", 100)\
#.option("startingOffsets", """{"IZkafkatopic.public.testiz":{"2":491437,"4":537712,"1":561234,"3":554189,"0":548887}}""") \

KAFKA_TOPIC_2_Write_2  = "IZkafkatopic2.public.testiz"

rawQuery = df \
.writeStream \
.format("kafka") \
.outputMode("append")\
.option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS_CONS) \
.option("topic", KAFKA_TOPIC_2_Write_2) \
.option("checkpointLocation", "/tmp/iverson/checkpointLocation/") \
.start()
#.awaitTermination()
    
    

print(rawQuery.status)
time.sleep(10)
print(rawQuery.status)
rawQuery.stop()

#rawQuery.awaitTermination()

#import threading  
  
#class thread(threading.Thread):  
#    def __init__(self, query):  
#        threading.Thread.__init__(self)  
#        self.query = query  
  
#        # helper function to execute the threads 
#    def run(self):  
#        print("progress ", query.lastProgress);  
        
#monitoring = thread(rawQuery)
#rawQuery.awaitTermination()       